In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import os
import glob
from collections import deque, defaultdict
import pandas as pd
import numpy as np

<IPython.core.display.Javascript object>

In [3]:
import warnings

# Suppress specific FutureWarning
warnings.filterwarnings("ignore", category=FutureWarning, module="seaborn._oldcore")


<IPython.core.display.Javascript object>

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns

<IPython.core.display.Javascript object>

In [5]:
from sklearn.preprocessing import StandardScaler

<IPython.core.display.Javascript object>

In [6]:
def read_csv_files_grouped(csv_files, plant, header=[0, 1]):
    results = []
    for filepath in csv_files:
        df = pd.read_csv(filepath, header=header, low_memory=False)
        df["plant"] = plant  # Add the 'plant' column
        results.append(df)
    df = pd.concat(results, ignore_index=True)
    return df

<IPython.core.display.Javascript object>

In [7]:
def read_csv_files_full(csv_files, plant):
    results = []
    for filepath in csv_files:
        df = pd.read_csv(filepath, low_memory=False)
        df["plant"] = plant  # Add the 'plant' column
        results.append(df)
    df = pd.concat(results, ignore_index=True)
    return df

<IPython.core.display.Javascript object>

### Renaming the Columns

In [8]:
def preprocess_rename_columns(df):
    df.columns = pd.MultiIndex.from_tuples(
        list(
            {
                col: (col[0], "") if "Unnamed" in col[1] else col for col in df.columns
            }.values()
        )
    )
    return df

<IPython.core.display.Javascript object>

### Changing the order_of columns Columns

In [9]:
def preprocess_change_columns_order(df, column, pos):
    # Get the list of columns
    cols = df.columns.tolist()
    # Remove the specified column
    cols.remove(column)
    # Insert the column at the desired position
    cols.insert(pos, column)
    # Reorder the DataFrame columns
    return df[cols]

<IPython.core.display.Javascript object>

In [10]:
def read_csv_files_path(csv_files_path_dict, path, plant):
    csv_files = glob.glob(os.path.join(path, "*.csv"))
    csv_files_path_dict[plant] = csv_files
    return csv_files_path_dict

<IPython.core.display.Javascript object>

# Reading The files

In [11]:
csv_files_path_grouped = dict()
csv_files_path_full = dict()

<IPython.core.display.Javascript object>

### 209

In [12]:
plants_209 = ["k"]

for plant in plants_209:
    path_grouped = f"/home/peressim/projects/ccs28-ml-modelling/reports/results/local_models/209/{plant}/grouped/"
    path_full = f"/home/peressim/projects/ccs28-ml-modelling/reports/results/local_models/209/{plant}/full/"
    csv_files_path_full = read_csv_files_path(csv_files_path_full, path_full, plant)
    csv_files_path_grouped = read_csv_files_path(
        csv_files_path_grouped, path_grouped, plant
    )

<IPython.core.display.Javascript object>

### Reading all data

#### Full

In [13]:
dfs_plant = dict()
column = ("plant", "")
pos = 2

for plant, csv_files in csv_files_path_full.items():
    df = read_csv_files_full(csv_files, plant)
    df["plant"] = plant
    dfs_plant[plant] = df

<IPython.core.display.Javascript object>

In [14]:
df_full = pd.concat(dfs_plant.values(), ignore_index=True)

<IPython.core.display.Javascript object>

In [15]:
df_full.shape

(6032, 23)

<IPython.core.display.Javascript object>

In [16]:
filter_df = df_full[df_full["R2 Test"] < 0].reset_index(drop=True)

filter_df["CP"] = (
    filter_df["Company"].astype(str) + "_" + filter_df["plant"].astype(str)
)

filter_df["CP"].unique()

print(filter_df.shape)

(0, 24)


<IPython.core.display.Javascript object>

In [17]:
patterns = [
    "Chemical + Mineralogical + Feature Engineering",
    "Chemical + Mineralogical + CS7 + One-Hot",
    "Chemical + Mineralogical + Physical + One-Hot",
    "Chemical + Mineralogical + CS3 + One-Hot",
    "Chemical + Mineralogical + CS3 + CS7 + One-Hot",
    "Chemical + Mineralogical + One-Hot",
    "Chemical + Mineralogical + CS1 + CS3 + CS7 + One-Hot",
    "Chemical + Mineralogical + CS1 + CS3 + One-Hot",
    "Chemical + Mineralogical + CS1 + One-Hot",
    "Chemical + Mineralogical + CS2 + One-Hot",
    "Chemical + Feature Engineering",
]

FEATURES_TO_REPLACE_1 = {
    "Chemical + Mineralogical + CS3 + CS7": "Chemical + Mineralogical + Early CS",
    "Chemical + Mineralogical + CS7": "Chemical + Mineralogical + Early CS",
    "Chemical + Mineralogical + CS3": "Chemical + Mineralogical + Early CS",
    "Chemical + Mineralogical + CS1": "Chemical + Mineralogical + Early CS",
    "Chemical + Mineralogical + CS1 + CS3": "Chemical + Mineralogical + Early CS",
    "Chemical + Mineralogical + CS1 + CS3 + CS7": "Chemical + Mineralogical + Early CS",
    "Chemical + Mineralogical + CS2": "Chemical + Mineralogical + Early CS",
    "Chemical + CS1 + CS3 + CS7": "Chemical + Mineralogical + Early CS",
    "Chemical + CS3 + CS7": "Chemical + Mineralogical + Early CS",
    "Chemical + CS7": "Chemical + Mineralogical + Early CS",
    "Chemical + CS3": "Chemical + Mineralogical + Early CS",
    "Chemical + CS1 + CS3": "Chemical + Mineralogical + Early CS",
    "Chemical + CS1": "Chemical + Mineralogical + Early CS",
    "Chemical + CS1 + CS7": "Chemical + Mineralogical + Early CS",
}

FEATURES_TO_REPLACE_2 = {
    "Chemical": "Chemical + Mineralogical",
    "Chemical + Properties CS Less": "Chemical + Mineralogical + Physical - Early CS",
    "Chemical + Mineralogical + Properties CS Less": "Chemical + Mineralogical + Physical - Early CS",
    "Chemical + Physical": "Chemical + Mineralogical + Physical",
}

df_full = df_full[~df_full["Features"].apply(lambda x: x in patterns)].reset_index(
    drop=True
)

df_full = df_full[~df_full["Cross Validation"].eq("Out of time Split")].reset_index(
    drop=True
)

# removing afterwards
df_full = df_full[~df_full["Cross Validation"].eq("Repeated KFold")].reset_index(
    drop=True
)

df_full["Features_bkp"] = df_full["Features"].copy()
df_full["Features"] = df_full["Features"].replace(FEATURES_TO_REPLACE_1)

df_full["Features"] = df_full["Features"].replace(FEATURES_TO_REPLACE_2)


# removing afterwards
df_full = df_full[
    ~df_full["Features"].eq("Chemical + Mineralogical + Early CS")
].reset_index(drop=True)

<IPython.core.display.Javascript object>

In [18]:
df_full["Cross Validation"].unique()

array(['Blocking Time Series Split', 'Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [19]:
df_full["Features"].unique()

array(['Chemical + Mineralogical + Physical - Early CS',
       'Chemical + Mineralogical + Physical', 'Chemical + Mineralogical'],
      dtype=object)

<IPython.core.display.Javascript object>

In [20]:
df_full["Features_bkp"].unique()

array(['Chemical + Properties CS Less', 'Chemical + Physical', 'Chemical'],
      dtype=object)

<IPython.core.display.Javascript object>

In [21]:
df_full.shape

(1560, 24)

<IPython.core.display.Javascript object>

In [22]:
df_full[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].describe().round(2).T

,count,mean,std,min,25%,50%,75%,max
RMSE Test,1560.0,1.96,0.79,0.56,1.41,1.80,2.32,5.81
MAE Test,1560.0,1.50,0.55,0.41,1.13,1.38,1.77,3.82
MAPE Test,1560.0,0.04,0.01,0.01,0.03,0.03,0.04,0.08
R2 Test,1560.0,0.87,0.12,0.26,0.83,0.91,0.95,0.99


<IPython.core.display.Javascript object>

In [23]:
df_full[df_full["R2 Test"] < 0][["Company", "plant"]]["plant"].nunique()

0

<IPython.core.display.Javascript object>

In [24]:
df_full[df_full["R2 Test"].eq(df_full["R2 Test"].min())]

,Category,Company,Plant,Features,Data Shape,Timesteps,Model,Model Params,Scaler,Scaler Params,...,RMSE Train,MAE Train,MAPE Train,R2 Train,RMSE Test,MAE Test,MAPE Test,R2 Test,plant,Features_bkp
104,Local Model,209,K,Chemical + Mineralogical + Physical,"(3345, 16)",1.0,LSTM,NaN,Standard Scaler,NaN,...,2.385923,1.816255,0.044026,0.799886,4.234359,3.356087,0.07891,0.257441,k,Chemical + Physical
109,Local Model,209,K,Chemical + Mineralogical + Physical,"(3345, 16)",1.0,LSTM,NaN,Standard Scaler,NaN,...,2.385923,1.816255,0.044026,0.799886,4.234359,3.356087,0.07891,0.257441,k,Chemical + Physical
114,Local Model,209,K,Chemical + Mineralogical + Physical,"(3345, 16)",1.0,LSTM,NaN,Standard Scaler,NaN,...,2.385923,1.816255,0.044026,0.799886,4.234359,3.356087,0.07891,0.257441,k,Chemical + Physical


<IPython.core.display.Javascript object>

In [25]:
df_full[(df_full["R2 Test"].eq(df_full["R2 Test"].min()))]["Cross Validation"]

104    Blocking Time Series Split
109    Blocking Time Series Split
114    Blocking Time Series Split
Name: Cross Validation, dtype: object

<IPython.core.display.Javascript object>

In [26]:
filter_df = df_full[df_full["R2 Test"] < 0].reset_index(drop=True)

print(filter_df.shape)

(0, 24)


<IPython.core.display.Javascript object>

In [27]:
df_full.shape

(1560, 24)

<IPython.core.display.Javascript object>

In [28]:
df_full["plant"].unique()

array(['k'], dtype=object)

<IPython.core.display.Javascript object>

In [29]:
df_full_209 = df_full[df_full["plant"].isin(["partner_i", "partner_ii", "partner_iv"])].reset_index(
    drop=True
)
df_full_209 = df_full[~df_full["plant"].isin(["partner_i", "partner_ii", "partner_iv"])].reset_index(
    drop=True
)

<IPython.core.display.Javascript object>

In [30]:
df_full.shape

(1560, 24)

<IPython.core.display.Javascript object>

In [31]:
df_full_209.shape

(1560, 24)

<IPython.core.display.Javascript object>

In [32]:
df_full_209.shape

(1560, 24)

<IPython.core.display.Javascript object>

In [33]:
df_full_209[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].describe().round(2).T

,count,mean,std,min,25%,50%,75%,max
RMSE Test,1560.0,1.96,0.79,0.56,1.41,1.80,2.32,5.81
MAE Test,1560.0,1.50,0.55,0.41,1.13,1.38,1.77,3.82
MAPE Test,1560.0,0.04,0.01,0.01,0.03,0.03,0.04,0.08
R2 Test,1560.0,0.87,0.12,0.26,0.83,0.91,0.95,0.99


<IPython.core.display.Javascript object>

In [34]:
df_full_209[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].describe().round(2).T

,count,mean,std,min,25%,50%,75%,max
RMSE Test,1560.0,1.96,0.79,0.56,1.41,1.80,2.32,5.81
MAE Test,1560.0,1.50,0.55,0.41,1.13,1.38,1.77,3.82
MAPE Test,1560.0,0.04,0.01,0.01,0.03,0.03,0.04,0.08
R2 Test,1560.0,0.87,0.12,0.26,0.83,0.91,0.95,0.99


<IPython.core.display.Javascript object>

# Analysis (full results)

In [35]:
df_full_209 = df_full[~df_full["plant"].isin(["partner_i", "partner_ii", "partner_iv"])].reset_index(
    drop=True
)

<IPython.core.display.Javascript object>

## 209

In [36]:
df_full_209.shape

(1560, 24)

<IPython.core.display.Javascript object>

In [37]:
df_full_209["Company"].unique()

array([209])

<IPython.core.display.Javascript object>

### Variable Grouping: CM

In [38]:
df_full_209["Features"].unique()

array(['Chemical + Mineralogical + Physical - Early CS',
       'Chemical + Mineralogical + Physical', 'Chemical + Mineralogical'],
      dtype=object)

<IPython.core.display.Javascript object>

In [39]:
df_results_full_cm = (
    df_full_209[df_full_209["Features"].eq("Chemical + Mineralogical")]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [40]:
df_results_full_cm["Features"].unique()

array(['Chemical + Mineralogical'], dtype=object)

<IPython.core.display.Javascript object>

In [41]:
df_results_full_cm.shape

(520, 24)

<IPython.core.display.Javascript object>

#### K-Fold

In [42]:
df_results_full_cm["Cross Validation"].unique()

array(['Blocking Time Series Split', 'Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [43]:
df_results_full_cm_kf = (
    df_results_full_cm[df_results_full_cm["Cross Validation"].eq("Repeated KFold")]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [44]:
df_results_full_cm_kf["Cross Validation"].unique()

array([], dtype=object)

<IPython.core.display.Javascript object>

In [45]:
df_results_full_cm_kf.shape

(0, 24)

<IPython.core.display.Javascript object>

In [46]:
df_results_full_cm_kf.groupby(["Company", "Model", "Timesteps"], dropna=False)[
    "RMSE Test"
].mean().shape

(0,)

<IPython.core.display.Javascript object>

### Blocking time series

In [47]:
df_results_full_cm["Cross Validation"].unique()

array(['Blocking Time Series Split', 'Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [48]:
df_results_full_cm_btss = (
    df_results_full_cm[
        df_results_full_cm["Cross Validation"].eq("Blocking Time Series Split")
    ]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [49]:
df_results_full_cm_btss["Cross Validation"].unique()

array(['Blocking Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [50]:
df_results_full_cm_btss.shape

(260, 24)

<IPython.core.display.Javascript object>

In [51]:
df_results_full_cm_btss.groupby(
    ["Company", "Plant", "Model", "Timesteps"], dropna=False
)["RMSE Test"].mean().shape

(20,)

<IPython.core.display.Javascript object>

### Time Series Split

In [52]:
df_results_full_cm["Cross Validation"].unique()

array(['Blocking Time Series Split', 'Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [53]:
df_results_full_cm_tss = (
    df_results_full_cm[df_results_full_cm["Cross Validation"].eq("Time Series Split")]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [54]:
df_results_full_cm_tss["Cross Validation"].unique()

array(['Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [55]:
df_results_full_cm_tss.shape

(260, 24)

<IPython.core.display.Javascript object>

In [56]:
df_results_full_cm_tss.groupby(
    ["Company", "Plant", "Model", "Timesteps"], dropna=False
)["RMSE Test"].mean().shape

(20,)

<IPython.core.display.Javascript object>

### Variable Grouping: CM-P

In [57]:
df_full_209["Features"].unique()

array(['Chemical + Mineralogical + Physical - Early CS',
       'Chemical + Mineralogical + Physical', 'Chemical + Mineralogical'],
      dtype=object)

<IPython.core.display.Javascript object>

In [58]:
df_results_full_cm_p = (
    df_full_209[
        df_full_209["Features"].eq("Chemical + Mineralogical + Physical - Early CS")
    ]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [59]:
df_results_full_cm_p["Features"].unique()

array(['Chemical + Mineralogical + Physical - Early CS'], dtype=object)

<IPython.core.display.Javascript object>

In [60]:
df_results_full_cm_p.shape

(520, 24)

<IPython.core.display.Javascript object>

### K-Fold

In [61]:
# df_results_full_cm_p["Cross Validation"].unique()

<IPython.core.display.Javascript object>

In [62]:
# df_results_full_cm_p_kf = (
#     df_results_full_cm_p[df_results_full_cm_p["Cross Validation"].eq("Repeated KFold")]
#     .copy()
#     .reset_index(drop=True)
# )

<IPython.core.display.Javascript object>

In [63]:
# df_results_full_cm_p_kf["Cross Validation"].unique()

<IPython.core.display.Javascript object>

In [64]:
# df_results_full_cm_p_kf.shape

<IPython.core.display.Javascript object>

In [65]:
# df_results_full_cm_kf.groupby(["Company", "Model", "Timesteps"], dropna=False)[
#     "RMSE Test"
# ].mean().shape

<IPython.core.display.Javascript object>

### Blocking time series

In [66]:
df_results_full_cm_p["Cross Validation"].unique()

array(['Blocking Time Series Split', 'Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [67]:
df_results_full_cm_p_btss = (
    df_results_full_cm_p[
        df_results_full_cm_p["Cross Validation"].eq("Blocking Time Series Split")
    ]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [68]:
df_results_full_cm_p_btss["Cross Validation"].unique()

array(['Blocking Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [69]:
df_results_full_cm_p_btss.shape

(260, 24)

<IPython.core.display.Javascript object>

In [70]:
df_results_full_cm_p_btss.shape, df_results_full_cm_btss.shape

((260, 24), (260, 24))

<IPython.core.display.Javascript object>

In [71]:
df_results_full_cm_p_btss.groupby(
    ["Company", "Cross Validation", "Plant", "Model", "Timesteps"], dropna=False
)["RMSE Test"].mean().shape

(20,)

<IPython.core.display.Javascript object>

In [72]:
df_results_full_cm_btss.shape, df_results_full_cm_p_btss.shape

df1 = df_results_full_cm_btss
df2 = df_results_full_cm_p_btss

grouping_columns = ["Company", "Plant", "Model", "Timesteps"]

<IPython.core.display.Javascript object>

In [73]:
# Group by and get sizes for both datasets
group_sizes_df1 = df1.groupby(grouping_columns, dropna=False).size()
group_sizes_df2 = df2.groupby(grouping_columns, dropna=False).size()

# Compare group sizes directly
group_size_diff = group_sizes_df1.compare(group_sizes_df2)
print("Differences in group sizes:")
print(group_size_diff)

Differences in group sizes:
Empty DataFrame
Columns: [self, other]
Index: []


<IPython.core.display.Javascript object>

In [74]:
# Group by and get sizes for both datasets
group_sizes_df1 = df1.groupby(grouping_columns, dropna=False).size()
group_sizes_df2 = df2.groupby(grouping_columns, dropna=False).size()

# Compare group sizes directly
group_size_diff = group_sizes_df2.compare(group_sizes_df1)
print("Differences in group sizes:")
print(group_size_diff)

Differences in group sizes:
Empty DataFrame
Columns: [self, other]
Index: []


<IPython.core.display.Javascript object>

### Time Series Split

In [75]:
df_results_full_cm_p["Cross Validation"].unique()

array(['Blocking Time Series Split', 'Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [76]:
df_results_full_cm_p_tss = (
    df_results_full_cm_p[
        df_results_full_cm_p["Cross Validation"].eq("Time Series Split")
    ]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [77]:
df_results_full_cm_p_tss["Cross Validation"].unique()

array(['Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [78]:
df_results_full_cm_p_tss.shape

(260, 24)

<IPython.core.display.Javascript object>

In [79]:
df_results_full_cm_p_tss.groupby(
    ["Company", "Model", "Plant", "Timesteps"], dropna=False
)["RMSE Test"].mean().shape

(20,)

<IPython.core.display.Javascript object>

### Variable Grouping: CM-P-CS

In [80]:
df_full_209["Features"].unique()

array(['Chemical + Mineralogical + Physical - Early CS',
       'Chemical + Mineralogical + Physical', 'Chemical + Mineralogical'],
      dtype=object)

<IPython.core.display.Javascript object>

In [81]:
df_results_full_cm_p_cs = (
    df_full_209[df_full_209["Features"].eq("Chemical + Mineralogical + Physical")]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [82]:
df_results_full_cm_p_cs["Features"].unique()

array(['Chemical + Mineralogical + Physical'], dtype=object)

<IPython.core.display.Javascript object>

In [83]:
df_results_full_cm_p_cs.shape

(520, 24)

<IPython.core.display.Javascript object>

### K-Fold

In [84]:
df_results_full_cm_p_cs["Cross Validation"].unique()

array(['Blocking Time Series Split', 'Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [85]:
df_results_full_cm_p_cs_kf = (
    df_results_full_cm_p_cs[
        df_results_full_cm_p_cs["Cross Validation"].eq("Repeated KFold")
    ]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [86]:
df_results_full_cm_p_cs_kf["Cross Validation"].unique()

array([], dtype=object)

<IPython.core.display.Javascript object>

In [87]:
df_results_full_cm_p_cs_kf.shape

(0, 24)

<IPython.core.display.Javascript object>

In [88]:
df_results_full_cm_p_cs_kf.groupby(["Company", "Model", "Timesteps"], dropna=False)[
    "RMSE Test"
].mean().shape

(0,)

<IPython.core.display.Javascript object>

### Blocking time series

In [89]:
df_results_full_cm_p_cs["Cross Validation"].unique()

array(['Blocking Time Series Split', 'Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [90]:
df_results_full_cm_p_cs_btss = (
    df_results_full_cm_p_cs[
        df_results_full_cm_p_cs["Cross Validation"].eq("Blocking Time Series Split")
    ]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [91]:
df_results_full_cm_p_cs_btss["Cross Validation"].unique()

array(['Blocking Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [92]:
df_results_full_cm_p_cs_btss.shape

(260, 24)

<IPython.core.display.Javascript object>

In [93]:
df_results_full_cm_p_cs_btss.groupby(
    ["Company", "Plant", "Model", "Timesteps"], dropna=False
)["RMSE Test"].mean().shape

(20,)

<IPython.core.display.Javascript object>

### Time Series Split

In [94]:
df_results_full_cm_p_cs["Cross Validation"].unique()

array(['Blocking Time Series Split', 'Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [95]:
df_results_full_cm_p_cs_tss = (
    df_results_full_cm_p_cs[
        df_results_full_cm_p_cs["Cross Validation"].eq("Time Series Split")
    ]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [96]:
df_results_full_cm_p_cs_tss["Cross Validation"].unique()

array(['Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [97]:
df_results_full_cm_p_cs_tss.shape

(260, 24)

<IPython.core.display.Javascript object>

In [98]:
df_results_full_cm_p_cs_tss.groupby(
    ["Company", "Plant", "Model", "Timesteps"], dropna=False
)["RMSE Test"].mean().shape

(20,)

<IPython.core.display.Javascript object>

# Grouped Dataset

In [99]:
dfs_plant = dict()
column = ("plant", "")
pos = 2

for plant, csv_files in csv_files_path_grouped.items():
    df = read_csv_files_grouped(csv_files, plant)
    df["plant"] = plant
    df = preprocess_rename_columns(df)
    df = preprocess_change_columns_order(df, column, pos)
    dfs_plant[plant] = df

<IPython.core.display.Javascript object>

In [100]:
df = pd.concat(dfs_plant)
df_copy = df.copy()
df_copy = (
    df_copy.reset_index(level=0)
    .rename({"level_0": "Plant"}, axis=1)
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [101]:
df_copy.shape

(554, 26)

<IPython.core.display.Javascript object>

In [102]:
df_full_filter = df_full[
    df_full["Cross Validation"].eq("Time Series Split")
].reset_index(drop=True)

df_full_groupped = df_full_filter.groupby(
    [
        "Category",
        "Company",
        "Features",
        "Data Shape",
        "Timesteps",
        "Model",
        "Cross Validation",
        "Cross Validation Params",
        "plant",
        "Plant",
    ],
    dropna=False,
    as_index=False,
).last()

<IPython.core.display.Javascript object>

In [103]:
df_copy = df_full_groupped

<IPython.core.display.Javascript object>

## Preprocessing steps

In [104]:
patterns = [
    "Chemical + Mineralogical + Feature Engineering",
    "Chemical + Mineralogical + CS7 + One-Hot",
    "Chemical + Mineralogical + Physical + One-Hot",
    "Chemical + Mineralogical + CS3 + One-Hot",
    "Chemical + Mineralogical + CS3 + CS7 + One-Hot",
    "Chemical + Mineralogical + One-Hot",
    "Chemical + Mineralogical + CS1 + CS3 + CS7 + One-Hot",
    "Chemical + Mineralogical + CS1 + CS3 + One-Hot",
    "Chemical + Mineralogical + CS1 + One-Hot",
    "Chemical + Mineralogical + CS2 + One-Hot",
    "Chemical + Feature Engineering",
]

replace_dict = {
    "Chemical + Mineralogical + CS3 + CS7": "Chemical + Mineralogical + Early CS",
    "Chemical + Mineralogical + CS7": "Chemical + Mineralogical + Early CS",
    "Chemical + Mineralogical + CS3": "Chemical + Mineralogical + Early CS",
    "Chemical + Mineralogical + CS1": "Chemical + Mineralogical + Early CS",
    "Chemical + Mineralogical + CS1 + CS3": "Chemical + Mineralogical + Early CS",
    "Chemical + Mineralogical + CS1 + CS3 + CS7": "Chemical + Mineralogical + Early CS",
    "Chemical + Mineralogical + CS2": "Chemical + Mineralogical + Early CS",
    "Chemical + CS1 + CS3 + CS7": "Chemical + Mineralogical + Early CS",
    "Chemical + CS3 + CS7": "Chemical + Mineralogical + Early CS",
    "Chemical + CS7": "Chemical + Mineralogical + Early CS",
    "Chemical + CS3": "Chemical + Mineralogical + Early CS",
    "Chemical + CS1 + CS3": "Chemical + Mineralogical + Early CS",
    "Chemical + CS1": "Chemical + Mineralogical + Early CS",
    "Chemical + CS1 + CS7": "Chemical + Mineralogical + Early CS",
}

replace_dict_2 = {
    "Chemical": "Chemical + Mineralogical",
    "Chemical + Properties CS Less": "Chemical + Mineralogical + Physical - Early CS",
    "Chemical + Mineralogical + Properties CS Less": "Chemical + Mineralogical + Physical - Early CS",
    "Chemical + Physical": "Chemical + Mineralogical + Physical",
}

df_copy = df_copy[~df_copy["Cross Validation"].eq("Out of time Split")].reset_index(
    drop=True
)

# Removing afterwards
df_copy = df_copy[~df_copy["Cross Validation"].eq("Repeated KFold")].reset_index(
    drop=True
)

df_copy = df_copy[~df_copy["Features"].apply(lambda x: x in patterns)].reset_index(
    drop=True
)
df_copy["Features_bkp"] = df_copy["Features"].copy()
df_copy["Features"] = df_copy["Features"].replace(replace_dict)

df_copy["Features"] = df_copy["Features"].replace(replace_dict_2)

# Removing afterwards
df_copy = df_copy[
    ~df_copy["Features"].eq("Chemical + Mineralogical + Early CS")
].reset_index(drop=True)

<IPython.core.display.Javascript object>

In [105]:
df_copy["MAPE Train"] = df_copy["MAPE Train"] * 100
df_copy["MAPE Test"] = df_copy["MAPE Test"] * 100

df_copy["Model_bkp"] = df_copy["Model"].copy()
df_copy["Model_bkp_2"] = df_copy["Model"] + df_copy["Timesteps"].apply(
    lambda x: "" if pd.isna(x) else "_" + str(x)
)
df_copy["Model"] = df_copy["Model"].replace(
    {
        "MLP": "Neural Networks",
        "LSTM": "Neural Networks",
        "GRU": "Neural Networks",
        "BidirectionalLSTM": "Neural Networks",
        "BidirectionalGRU": "Neural Networks",
        "Transformer": "Neural Networks",
        "Decision Tree": "Trees",
        "Random Forest": "Trees",
        "XGBoost": "Trees",
    }
)

<IPython.core.display.Javascript object>

In [106]:
df_copy["Cross Validation"].unique()

array(['Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [107]:
df_copy["Features"].unique()

array(['Chemical + Mineralogical', 'Chemical + Mineralogical + Physical',
       'Chemical + Mineralogical + Physical - Early CS'], dtype=object)

<IPython.core.display.Javascript object>

In [108]:
df_copy.shape

(60, 26)

<IPython.core.display.Javascript object>

In [109]:
df_copy["plant"].unique()

array(['k'], dtype=object)

<IPython.core.display.Javascript object>

In [110]:
df_full.shape

(1560, 24)

<IPython.core.display.Javascript object>

In [111]:
cols_groupby = [
    # "Category",
    # "Company",
    "plant",
    # "Data Shape",
    "Timesteps",
    "Features_bkp",
    "Model",
    "Cross Validation",
    "Cross Validation Params",
]

cols_agg = ["RMSE Train", "MAE Train", "MAPE Train", "R2 Train"] + [
    "RMSE Test",
    "MAE Test",
    "MAPE Test",
    "R2 Test",
]

df_results_to_save = (
    df_full.groupby(cols_groupby, dropna=False)[cols_agg]
    .agg(["mean", lambda series: pd.Series(series.std(ddof=0), name="std")])
    .reset_index()
    .rename(columns={"<lambda_0>": "std"})
)

<IPython.core.display.Javascript object>

In [112]:
df_results_to_save.shape

(120, 22)

<IPython.core.display.Javascript object>

In [113]:
df_copy.shape

(60, 26)

<IPython.core.display.Javascript object>

In [114]:
df_copy_grouped = df_copy.copy()

<IPython.core.display.Javascript object>

In [115]:
df_copy_grouped.shape

(60, 26)

<IPython.core.display.Javascript object>

# Topological Analysis - NEW

## Definitions

In [116]:
# Function to compare two models strictly based on means only
def dominates(model1, model2):
    """Return True if model1 strictly dominates model2, False otherwise."""
    # Check if model1 is at least as good as model2 in all metrics
    at_least_as_good = (
        model1["RMSE_mean"] <= model2["RMSE_mean"]
        and model1["MAE_mean"] <= model2["MAE_mean"]
        and model1["MAPE_mean"] <= model2["MAPE_mean"]
        and model1["R2_mean"] >= model2["R2_mean"]
    )

    # Check if model1 is strictly better in at least one metric
    strictly_better_in_one = (
        model1["RMSE_mean"] < model2["RMSE_mean"]
        or model1["MAE_mean"] < model2["MAE_mean"]
        or model1["MAPE_mean"] < model2["MAPE_mean"]
        or model1["R2_mean"] > model2["R2_mean"]
    )

    # Model1 strictly dominates Model2 if it is at least as good in all metrics and
    # strictly better in at least one metric
    return at_least_as_good and strictly_better_in_one

<IPython.core.display.Javascript object>

In [117]:
def get_dominance_matrix_and_graph(df):
    n = len(df)
    dominance_matrix = np.zeros((n, n), dtype=bool)

    for i in range(n):
        for j in range(n):
            if i != j:
                dominance_matrix[i, j] = dominates(df.iloc[i], df.iloc[j])

    # Create the dominance graph
    dominance_graph = {i: [] for i in range(n)}
    for i in range(n):
        for j in range(n):
            if dominance_matrix[i, j]:
                dominance_graph[i].append(j)

    return dominance_matrix, dominance_graph

<IPython.core.display.Javascript object>

In [118]:
# Topological Sorting
def topological_sort(graph):
    """Perform topological sorting on the given graph."""
    in_degree = {u: 0 for u in graph}
    for u in graph:
        for v in graph[u]:
            in_degree[v] += 1

    queue = deque([u for u in graph if in_degree[u] == 0])
    topo_order = []

    while queue:
        u = queue.popleft()
        topo_order.append(u)

        for v in graph[u]:
            in_degree[v] -= 1
            if in_degree[v] == 0:
                queue.append(v)

    return topo_order

<IPython.core.display.Javascript object>

In [119]:
# Function to find the top models in each group
def find_top_models(group):
    max_net_dominance = group["Net_Dominance"].max()
    top_models = group[group["Net_Dominance"] == max_net_dominance]
    return top_models

<IPython.core.display.Javascript object>

### Dominance Analysis and SCPM

In [120]:
def compute_scpm(df):
    scaler = StandardScaler()
    standardized_metrics = scaler.fit_transform(
        df[["RMSE_mean", "MAE_mean", "MAPE_mean", "R2_mean"]]
    )

    # Creating a new dataframe with standardized values
    standardized_df = pd.DataFrame(
        standardized_metrics,
        columns=["RMSE_mean", "MAE_mean", "MAPE_mean", "R2_mean"],
    )

    # Summing all metrics and subtracting R2_mean
    standardized_df["Result"] = (
        standardized_df[["RMSE_mean", "MAE_mean", "MAPE_mean"]].sum(axis=1)
        - standardized_df["R2_mean"]
    )

    df["SCPM"] = standardized_df["Result"]
    return df

<IPython.core.display.Javascript object>

In [121]:
def make_dominance_analysis(df):
    # Compute dominance matrix and graph
    dominance_matrix, dominance_graph = get_dominance_matrix_and_graph(df)

    # Get the topological order
    topo_order = topological_sort(dominance_graph)
    df = df.copy()
    n = len(topo_order)

    # Identify Dominant Models
    dominant_models = [
        df["Model_bkp_2"].iloc[i]
        for i in range(len(df))
        if not np.any(dominance_matrix[:, i])
    ]

    # Identify Non-Dominant Models
    non_dominant_models = [
        df["Model_bkp_2"].iloc[i]
        for i in range(len(df))
        if np.any(dominance_matrix[:, i])
    ]

    # Identify Models that are both dominated and dominate at least one
    intermediate_models = [
        df["Model_bkp_2"].iloc[i]
        for i in range(len(df))
        if np.any(dominance_matrix[:, i]) and np.any(dominance_matrix[i, :])
    ]

    # Add count of times each model is dominated by others
    df["Dominated_Count"] = np.sum(dominance_matrix, axis=0)

    # Classify models
    df["Classification"] = "intermediate_model"  # default to intermediate_model
    df.loc[
        ~np.any(dominance_matrix, axis=1) & np.any(dominance_matrix, axis=0),
        "Classification",
    ] = "non_dominant_model"
    df.loc[~np.any(dominance_matrix, axis=0), "Classification"] = "dominant_model"

    # Map the topological order to the model names and get dominance counts
    sorted_models = [df["Model"].iloc[i] for i in topo_order]

    df_sorted = df.iloc[topo_order].copy()
    df_sorted["Dominates_Count"] = dominance_matrix.sum(axis=1)[topo_order]
    df_sorted_topo = df_sorted.copy()

    df_sorted = df_sorted.sort_values(by="Dominates_Count", ascending=False)
    df_sorted_count = df_sorted.copy()

    return {
        "dominance_matrix": dominance_matrix,
        "dominance_graph": dominance_graph,
        "df_sorted_topo": df_sorted_topo,
        "df_sorted_count": df_sorted_count,
    }

<IPython.core.display.Javascript object>

# Analysis

## 209

In [122]:
# Filter only 209 project
df_209_grouped = df_copy_grouped[
    ~df_copy_grouped["plant"].isin(["partner_i", "partner_ii", "partner_iv"])
]

<IPython.core.display.Javascript object>

In [123]:
df_results = df_209_grouped[
    [
        ("Company"),
        ("plant"),
        ("Cross Validation"),
        ("Features"),
        ("Features_bkp"),
        ("Model"),
        ("Model_bkp"),
        ("Model_bkp_2"),
        ("Timesteps"),
        ("RMSE Test"),
        ("MAE Test"),
        ("MAPE Test"),
        ("R2 Test"),
    ]
].copy()
df_results = df_results.reset_index(drop=True)

# Rename columns
new_column_names = [
    "Company",
    "Plant",
    "Cross Validation",
    "Features",
    "Features_bkp",
    "Model",
    "Model_bkp",
    "Model_bkp_2",
    "Timesteps",
    "RMSE_mean",
    "MAE_mean",
    "MAPE_mean",
    "R2_mean",
]

df_results.columns = new_column_names

<IPython.core.display.Javascript object>

In [124]:
df_results.shape

(60, 13)

<IPython.core.display.Javascript object>

In [125]:
df_results["Cross Validation"].unique()

array(['Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

## Variable Grouping: CM

In [126]:
df_results["Features"].unique()

array(['Chemical + Mineralogical', 'Chemical + Mineralogical + Physical',
       'Chemical + Mineralogical + Physical - Early CS'], dtype=object)

<IPython.core.display.Javascript object>

In [127]:
df_results_cm = (
    df_results[df_results["Features"].eq("Chemical + Mineralogical")]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [128]:
df_results_cm["Features"].unique()

array(['Chemical + Mineralogical'], dtype=object)

<IPython.core.display.Javascript object>

In [129]:
df_results_cm.shape

(20, 13)

<IPython.core.display.Javascript object>

### Time Series Split

In [130]:
df_results_cm["Cross Validation"].unique()

array(['Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [131]:
df_results_cm_tss = (
    df_results_cm[df_results_cm["Cross Validation"].eq("Time Series Split")]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [132]:
df_results_cm_tss["Cross Validation"].unique()

array(['Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [133]:
df_results_cm_tss.shape

(20, 13)

<IPython.core.display.Javascript object>

###### SCPM computation

In [134]:
df_results_cm_tss = compute_scpm(df_results_cm_tss)

<IPython.core.display.Javascript object>

##### Dominance Analysis

In [135]:
# Per project
dominance_dict = make_dominance_analysis(df_results_cm_tss)
dominance_matrix_cm_tss = dominance_dict["dominance_matrix"]
dominance_graph_cm_tss = dominance_dict["dominance_graph"]
df_sorted_count = dominance_dict["df_sorted_count"]
df_sorted_topo = dominance_dict["df_sorted_topo"]

<IPython.core.display.Javascript object>

In [136]:
dominance_matrix_cm_tss.shape, len(dominance_graph_cm_tss)

((20, 20), 20)

<IPython.core.display.Javascript object>

In [137]:
df_sorted_topo["Classification"].value_counts()

Classification
intermediate_model    18
dominant_model         1
non_dominant_model     1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [138]:
df_sorted_topo.to_csv(
    "../../../../reports/results/local_models/ecics/209_k_dominance_analysis_cm_tss_last_fold.csv",
    index=False,
)

<IPython.core.display.Javascript object>

In [139]:
df_sorted_topo.head(5)

,Company,Plant,Cross Validation,Features,Features_bkp,Model,Model_bkp,Model_bkp_2,Timesteps,RMSE_mean,MAE_mean,MAPE_mean,R2_mean,SCPM,Dominated_Count,Classification,Dominates_Count
8,209,k,Time Series Split,Chemical + Mineralogical,Chemical + Mineralogical,Neural Networks,LSTM,LSTM_7.0,7.0,1.794403,1.400231,3.327944,0.884003,-4.446340,0,dominant_model,19
5,209,k,Time Series Split,Chemical + Mineralogical,Chemical + Mineralogical,Neural Networks,BidirectionalGRU,BidirectionalGRU_7.0,7.0,1.873674,1.431434,3.360627,0.873528,-4.020636,1,intermediate_model,18
6,209,k,Time Series Split,Chemical + Mineralogical,Chemical + Mineralogical,Neural Networks,BidirectionalLSTM,BidirectionalLSTM_7.0,7.0,1.959681,1.447141,3.388301,0.861651,-3.609196,2,intermediate_model,16
7,209,k,Time Series Split,Chemical + Mineralogical,Chemical + Mineralogical,Neural Networks,GRU,GRU_7.0,7.0,1.910072,1.516847,3.621194,0.868566,-3.368125,2,intermediate_model,14
4,209,k,Time Series Split,Chemical + Mineralogical,Chemical + Mineralogical,Neural Networks,Transformer,Transformer_1.0,1.0,2.097326,1.464751,3.430982,0.842025,-2.964986,3,intermediate_model,14


<IPython.core.display.Javascript object>

In [140]:
df_sorted_topo_cols = df_sorted_topo[
    [
        # "Model",
        # "Model_bkp",
        "Model_bkp_2",
        "RMSE_mean",
        "MAE_mean",
        "MAPE_mean",
        "R2_mean",
        "SCPM",
        "Dominated_Count",
        "Classification",
        "Dominates_Count",
        "Plant",
    ]
].round(2)

<IPython.core.display.Javascript object>

###### Dominant Models

In [141]:
# Top 5
df_sorted_topo_cols.iloc[0:5].sort_values(
    by=["Dominates_Count", "SCPM"], ascending=[False, True]
).round(2)

,Model_bkp_2,RMSE_mean,MAE_mean,MAPE_mean,R2_mean,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
8,LSTM_7.0,1.79,1.40,3.33,0.88,-4.45,0,dominant_model,19,k
5,BidirectionalGRU_7.0,1.87,1.43,3.36,0.87,-4.02,1,intermediate_model,18,k
6,BidirectionalLSTM_7.0,1.96,1.45,3.39,0.86,-3.61,2,intermediate_model,16,k
7,GRU_7.0,1.91,1.52,3.62,0.87,-3.37,2,intermediate_model,14,k
4,Transformer_1.0,2.10,1.46,3.43,0.84,-2.96,3,intermediate_model,14,k


<IPython.core.display.Javascript object>

In [142]:
df_sorted_topo_cols[df_sorted_topo_cols["Classification"].eq("dominant_model")].shape

(1, 10)

<IPython.core.display.Javascript object>

###### Intermediate models

In [143]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("intermediate_model")
].sort_values(by=["Dominates_Count", "SCPM"], ascending=[False, True]).iloc[:4].round(2)

,Model_bkp_2,RMSE_mean,MAE_mean,MAPE_mean,R2_mean,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
5,BidirectionalGRU_7.0,1.87,1.43,3.36,0.87,-4.02,1,intermediate_model,18,k
6,BidirectionalLSTM_7.0,1.96,1.45,3.39,0.86,-3.61,2,intermediate_model,16,k
7,GRU_7.0,1.91,1.52,3.62,0.87,-3.37,2,intermediate_model,14,k
4,Transformer_1.0,2.10,1.46,3.43,0.84,-2.96,3,intermediate_model,14,k


<IPython.core.display.Javascript object>

In [144]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("intermediate_model")
].shape

(18, 10)

<IPython.core.display.Javascript object>

###### non-dominant models

In [145]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("non_dominant_model")
].sort_values(by=["Dominates_Count", "SCPM"], ascending=[False, True]).iloc[:3]

,Model_bkp_2,RMSE_mean,MAE_mean,MAPE_mean,R2_mean,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
16,Linear Regression,3.58,2.96,7.03,0.54,11.96,19,non_dominant_model,0,k


<IPython.core.display.Javascript object>

In [146]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("non_dominant_model")
].shape

(1, 10)

<IPython.core.display.Javascript object>

###### Statistics per model type

In [147]:
df_sorted_topo_models_cols = df_sorted_topo[
    [
        "Model",
        # "Model_bkp",
        # "Model_bkp_2",
        "RMSE_mean",
        "MAE_mean",
        "MAPE_mean",
        "R2_mean",
        "SCPM",
        "Dominated_Count",
        "Dominates_Count",
        "Classification",
        "Plant",
    ]
]  # .round(2)

<IPython.core.display.Javascript object>

In [148]:
df_sorted_topo["Model"].value_counts()

Model
Neural Networks      16
Trees                 3
Linear Regression     1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [149]:
df_sorted_topo_models_grouped = df_sorted_topo_models_cols.groupby("Model").describe().T

<IPython.core.display.Javascript object>

In [150]:
df_sorted_topo_models_cols = df_sorted_topo[
    [
        "Model",
        "Dominated_Count",
        "Dominates_Count",
        "Classification",
    ]
]

<IPython.core.display.Javascript object>

In [151]:
(
    df_sorted_topo_models_cols.groupby("Model")[["Classification"]]
    .value_counts(normalize=True)
    .to_frame()
    * 100
).round(2)

proportion
Model             Classification                
Linear Regression non_dominant_model      100.00
Neural Networks   intermediate_model       93.75
                  dominant_model            6.25
Trees             intermediate_model      100.00

<IPython.core.display.Javascript object>

In [152]:
df_sorted_topo_models_cols.groupby("Model").describe().round(2).T

Model                  Linear Regression  Neural Networks  Trees
Dominated_Count count                1.0            16.00   3.00
                mean                19.0             7.88   6.67
                std                  NaN             6.18   2.52
                min                 19.0             0.00   4.00
                25%                 19.0             2.75   5.50
                50%                 19.0             6.50   7.00
                75%                 19.0            14.25   8.00
                max                 19.0            18.00   9.00
Dominates_Count count                1.0            16.00   3.00
                mean                 0.0             9.12   6.33
                std                  NaN             5.76   0.58
                min                  0.0             1.00   6.00
                25%                  0.0             4.75   6.00
                50%                  0.0             8.50   6.00
                75%                  0.0            14.00   6.50
                max                  0.0            19.00   7.00

<IPython.core.display.Javascript object>

In [153]:
# df_sorted_topo_models_cols

<IPython.core.display.Javascript object>

In [154]:
df_sorted_topo_models_cols.groupby("Model")["Classification"].describe()

,count,unique,top,freq
Model,,,,
Linear Regression,1,1,non_dominant_model,1
Neural Networks,16,2,intermediate_model,15
Trees,3,1,intermediate_model,3


<IPython.core.display.Javascript object>

In [155]:
df_sorted_topo_models_cols.groupby("Model")[["Classification"]].value_counts()

Model              Classification    
Linear Regression  non_dominant_model     1
Neural Networks    intermediate_model    15
                   dominant_model         1
Trees              intermediate_model     3
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [156]:
summary_stats = df_sorted_topo_models_cols.groupby("Model").agg(
    Total_Models=("Model", "count"),
    Mean_Dominance_Count=("Dominates_Count", "mean"),
    Median_Dominance_Count=("Dominates_Count", "median"),
    Total_Dominated=("Dominated_Count", "sum"),
    Total_Dominating=("Dominates_Count", "sum"),
)

total_models = summary_stats[
    "Total_Models"
].sum()  # Get the total number of models across all categories
summary_stats["Dominance Proportion"] = summary_stats["Total_Dominating"] / total_models

<IPython.core.display.Javascript object>

In [157]:
summary_stats.round(2)

,Total_Models,Mean_Dominance_Count,Median_Dominance_Count,Total_Dominated,Total_Dominating,Dominance Proportion
Model,,,,,,
Linear Regression,1,0.00,0.0,19,0,0.00
Neural Networks,16,9.12,8.5,126,146,7.30
Trees,3,6.33,6.0,20,19,0.95


<IPython.core.display.Javascript object>

In [158]:
df_sorted_topo_models_grouped.T

RMSE_mean                                                    \
                      count      mean       std       min       25%       50%   
Model                                                                           
Linear Regression       1.0  3.579859       NaN  3.579859  3.579859  3.579859   
Neural Networks        16.0  2.301683  0.392801  1.794403  2.022485  2.241580   
Trees                   3.0  2.409216  0.080092  2.325665  2.371159  2.416653   

                                      MAE_mean            ... Dominated_Count  \
                        75%       max    count      mean  ...             75%   
Model                                                     ...                   
Linear Regression  3.579859  3.579859      1.0  2.964337  ...           19.00   
Neural Networks    2.581940  3.029273     16.0  1.732019  ...           14.25   
Trees              2.450992  2.485330      3.0  1.605120  ...            8.00   

                        Dominates_Count                                      \
                    max           count      mean       std  min   25%  50%   
Model                                                                         
Linear Regression  19.0             1.0  0.000000       NaN  0.0  0.00  0.0   
Neural Networks    18.0            16.0  9.125000  5.760498  1.0  4.75  8.5   
Trees               9.0             3.0  6.333333  0.577350  6.0  6.00  6.0   

                               
                    75%   max  
Model                          
Linear Regression   0.0   0.0  
Neural Networks    14.0  19.0  
Trees               6.5   7.0  

[3 rows x 56 columns]

<IPython.core.display.Javascript object>

### SCPM Analysis

In [159]:
cols = [
    # "Model",
    # "Model_bkp",
    "Model_bkp_2",
    "RMSE_mean",
    "MAE_mean",
    "MAPE_mean",
    "R2_mean",
    "SCPM",
    "Dominated_Count",
    "Dominates_Count",
    "Classification",
    "Plant",
]

<IPython.core.display.Javascript object>

#### SCPM Analysis by Project

In [160]:
df_sorted_scpm = df_sorted_topo.sort_values(by="SCPM").copy()

<IPython.core.display.Javascript object>

In [161]:
# (df_sorted_topo[cols].reset_index() == df_sorted_scpm[cols].reset_index()).all(axis=1)
df_sorted_scpm["topo_scmp_order_eq"] = df_sorted_topo.index == df_sorted_scpm.index

<IPython.core.display.Javascript object>

In [162]:
df_sorted_scpm[cols + ["topo_scmp_order_eq"]].round(2).head(5)

,Model_bkp_2,RMSE_mean,MAE_mean,MAPE_mean,R2_mean,SCPM,Dominated_Count,Dominates_Count,Classification,Plant,topo_scmp_order_eq
8,LSTM_7.0,1.79,1.40,3.33,0.88,-4.45,0,19,dominant_model,k,True
5,BidirectionalGRU_7.0,1.87,1.43,3.36,0.87,-4.02,1,18,intermediate_model,k,True
6,BidirectionalLSTM_7.0,1.96,1.45,3.39,0.86,-3.61,2,16,intermediate_model,k,True
7,GRU_7.0,1.91,1.52,3.62,0.87,-3.37,2,14,intermediate_model,k,True
4,Transformer_1.0,2.10,1.46,3.43,0.84,-2.96,3,14,intermediate_model,k,True


<IPython.core.display.Javascript object>

### Variable Grouping: CM-P

In [163]:
df_results["Features"].unique()

array(['Chemical + Mineralogical', 'Chemical + Mineralogical + Physical',
       'Chemical + Mineralogical + Physical - Early CS'], dtype=object)

<IPython.core.display.Javascript object>

In [164]:
df_results_cm_p = (
    df_results[
        df_results["Features"].eq("Chemical + Mineralogical + Physical - Early CS")
    ]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [165]:
df_results_cm_p["Features"].unique()

array(['Chemical + Mineralogical + Physical - Early CS'], dtype=object)

<IPython.core.display.Javascript object>

In [166]:
df_results_cm_p.shape

(20, 13)

<IPython.core.display.Javascript object>

### Time Series Split

In [167]:
df_results_cm_p["Cross Validation"].unique()

array(['Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [168]:
df_results_cm_p_tss = (
    df_results_cm_p[df_results_cm_p["Cross Validation"].eq("Time Series Split")]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [169]:
df_results_cm_p_tss["Cross Validation"].unique()

array(['Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [170]:
df_results_cm_p_tss.shape

(20, 13)

<IPython.core.display.Javascript object>

###### SCPM computation

In [171]:
df_results_cm_p_tss = compute_scpm(df_results_cm_p_tss)

<IPython.core.display.Javascript object>

##### Dominance Analysis

In [172]:
# Per project
dominance_dict = make_dominance_analysis(df_results_cm_p_tss)
dominance_matrix_cm_p_tss = dominance_dict["dominance_matrix"]
dominance_graph_cm_p_tss = dominance_dict["dominance_graph"]
df_sorted_count = dominance_dict["df_sorted_count"]
df_sorted_topo = dominance_dict["df_sorted_topo"]

<IPython.core.display.Javascript object>

In [173]:
dominance_matrix_cm_p_tss.shape, len(dominance_graph_cm_p_tss)

((20, 20), 20)

<IPython.core.display.Javascript object>

In [174]:
df_sorted_topo["Classification"].value_counts()

Classification
intermediate_model    16
non_dominant_model     3
dominant_model         1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [175]:
df_sorted_topo.to_csv(
    "../../../../reports/results/local_models/ecics/209_k_dominance_analysis_cm_p_tss_last_fold.csv",
    index=False,
)

<IPython.core.display.Javascript object>

In [176]:
df_sorted_topo.head(5)

,Company,Plant,Cross Validation,Features,Features_bkp,Model,Model_bkp,Model_bkp_2,Timesteps,RMSE_mean,MAE_mean,MAPE_mean,R2_mean,SCPM,Dominated_Count,Classification,Dominates_Count
4,209,k,Time Series Split,Chemical + Mineralogical + Physical - Early CS,Chemical + Mineralogical + Physical - Early CS,Neural Networks,Transformer,Transformer_1.0,1.0,1.601813,1.197847,2.862720,0.907853,-5.024111,0,dominant_model,19
12,209,k,Time Series Split,Chemical + Mineralogical + Physical - Early CS,Chemical + Mineralogical + Physical - Early CS,Neural Networks,GRU,GRU_14.0,14.0,1.642473,1.259062,2.956214,0.902552,-4.228164,1,intermediate_model,17
16,209,k,Time Series Split,Chemical + Mineralogical + Physical - Early CS,Chemical + Mineralogical + Physical - Early CS,Linear Regression,Linear Regression,Linear Regression,NaN,1.659827,1.210128,2.939928,0.901058,-4.428455,1,intermediate_model,17
0,209,k,Time Series Split,Chemical + Mineralogical + Physical - Early CS,Chemical + Mineralogical + Physical - Early CS,Neural Networks,BidirectionalGRU,BidirectionalGRU_1.0,1.0,1.742754,1.310853,3.113844,0.890924,-2.985992,3,intermediate_model,16
2,209,k,Time Series Split,Chemical + Mineralogical + Physical - Early CS,Chemical + Mineralogical + Physical - Early CS,Neural Networks,GRU,GRU_1.0,1.0,1.809294,1.322326,3.134193,0.882436,-2.448291,4,intermediate_model,10


<IPython.core.display.Javascript object>

In [177]:
df_sorted_topo_cols = df_sorted_topo[
    [
        # "Model",
        # "Model_bkp",
        "Model_bkp_2",
        "RMSE_mean",
        "MAE_mean",
        "MAPE_mean",
        "R2_mean",
        "SCPM",
        "Dominated_Count",
        "Classification",
        "Dominates_Count",
        "Plant",
    ]
].round(2)

<IPython.core.display.Javascript object>

###### Dominant Models

In [178]:
# Top 5
df_sorted_topo_cols.sort_values(
    by=["Dominates_Count", "SCPM"], ascending=[False, True]
).iloc[0:5].round(2)

,Model_bkp_2,RMSE_mean,MAE_mean,MAPE_mean,R2_mean,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
4,Transformer_1.0,1.60,1.20,2.86,0.91,-5.02,0,dominant_model,19,k
16,Linear Regression,1.66,1.21,2.94,0.90,-4.43,1,intermediate_model,17,k
12,GRU_14.0,1.64,1.26,2.96,0.90,-4.23,1,intermediate_model,17,k
0,BidirectionalGRU_1.0,1.74,1.31,3.11,0.89,-2.99,3,intermediate_model,16,k
13,LSTM_14.0,1.75,1.33,3.12,0.89,-2.78,4,intermediate_model,13,k


<IPython.core.display.Javascript object>

In [179]:
# Top 4
df_sorted_topo_cols.iloc[0:5].sort_values(by="Dominates_Count", ascending=False)

,Model_bkp_2,RMSE_mean,MAE_mean,MAPE_mean,R2_mean,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
4,Transformer_1.0,1.60,1.20,2.86,0.91,-5.02,0,dominant_model,19,k
12,GRU_14.0,1.64,1.26,2.96,0.90,-4.23,1,intermediate_model,17,k
16,Linear Regression,1.66,1.21,2.94,0.90,-4.43,1,intermediate_model,17,k
0,BidirectionalGRU_1.0,1.74,1.31,3.11,0.89,-2.99,3,intermediate_model,16,k
2,GRU_1.0,1.81,1.32,3.13,0.88,-2.45,4,intermediate_model,10,k


<IPython.core.display.Javascript object>

In [180]:
df_sorted_topo_cols[df_sorted_topo_cols["Classification"].eq("dominant_model")].shape

(1, 10)

<IPython.core.display.Javascript object>

###### Intermediate models

In [181]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("intermediate_model")
].sort_values(by=["Dominates_Count", "SCPM"], ascending=[False, True]).iloc[:3]

,Model_bkp_2,RMSE_mean,MAE_mean,MAPE_mean,R2_mean,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
16,Linear Regression,1.66,1.21,2.94,0.90,-4.43,1,intermediate_model,17,k
12,GRU_14.0,1.64,1.26,2.96,0.90,-4.23,1,intermediate_model,17,k
0,BidirectionalGRU_1.0,1.74,1.31,3.11,0.89,-2.99,3,intermediate_model,16,k


<IPython.core.display.Javascript object>

In [182]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("intermediate_model")
].shape

(16, 10)

<IPython.core.display.Javascript object>

###### non-dominant models

In [183]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("non_dominant_model")
].sort_values(by=["Dominates_Count", "SCPM"], ascending=[False, True]).iloc[:5]

,Model_bkp_2,RMSE_mean,MAE_mean,MAPE_mean,R2_mean,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
8,LSTM_7.0,2.32,1.82,4.30,0.81,6.33,16,non_dominant_model,0,k
15,Decision Tree,2.62,1.65,3.91,0.75,6.87,16,non_dominant_model,0,k
7,GRU_7.0,2.28,1.91,4.62,0.81,7.20,15,non_dominant_model,0,k


<IPython.core.display.Javascript object>

In [184]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("non_dominant_model")
].shape

(3, 10)

<IPython.core.display.Javascript object>

###### Statistics per model type

In [185]:
df_sorted_topo_models_cols = df_sorted_topo[
    [
        "Model",
        # "Model_bkp",
        # "Model_bkp_2",
        "RMSE_mean",
        "MAE_mean",
        "MAPE_mean",
        "R2_mean",
        "SCPM",
        "Dominated_Count",
        "Dominates_Count",
        "Classification",
        "Plant",
    ]
]  # .round(2)

<IPython.core.display.Javascript object>

In [186]:
df_sorted_topo["Model"].value_counts()

Model
Neural Networks      16
Trees                 3
Linear Regression     1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [187]:
df_sorted_topo_models_grouped = df_sorted_topo_models_cols.groupby("Model").describe().T

<IPython.core.display.Javascript object>

In [188]:
df_sorted_topo_models_cols = df_sorted_topo[
    [
        "Model",
        "Dominated_Count",
        "Dominates_Count",
        "Classification",
    ]
]

<IPython.core.display.Javascript object>

In [189]:
(
    df_sorted_topo_models_cols.groupby("Model")[["Classification"]]
    .value_counts(normalize=True)
    .to_frame()
    * 100
).round(2)

proportion
Model             Classification                
Linear Regression intermediate_model      100.00
Neural Networks   intermediate_model       81.25
                  non_dominant_model       12.50
                  dominant_model            6.25
Trees             intermediate_model       66.67
                  non_dominant_model       33.33

<IPython.core.display.Javascript object>

In [190]:
df_sorted_topo_models_cols.groupby("Model").describe().round(2).T

Model                  Linear Regression  Neural Networks  Trees
Dominated_Count count                1.0            16.00   3.00
                mean                 1.0             7.06  15.00
                std                  NaN             4.97   1.00
                min                  1.0             0.00  14.00
                25%                  1.0             4.00  14.50
                50%                  1.0             5.50  15.00
                75%                  1.0            10.00  15.50
                max                  1.0            16.00  16.00
Dominates_Count count                1.0            16.00   3.00
                mean                17.0             8.75   0.67
                std                  NaN             5.42   0.58
                min                 17.0             0.00   0.00
                25%                 17.0             6.75   0.50
                50%                 17.0             8.00   1.00
                75%                 17.0            10.75   1.00
                max                 17.0            19.00   1.00

<IPython.core.display.Javascript object>

In [191]:
# df_sorted_topo_models_cols

<IPython.core.display.Javascript object>

In [192]:
df_sorted_topo_models_cols.groupby("Model")["Classification"].describe()

,count,unique,top,freq
Model,,,,
Linear Regression,1,1,intermediate_model,1
Neural Networks,16,3,intermediate_model,13
Trees,3,2,intermediate_model,2


<IPython.core.display.Javascript object>

In [193]:
df_sorted_topo_models_cols.groupby("Model")[["Classification"]].value_counts()

Model              Classification    
Linear Regression  intermediate_model     1
Neural Networks    intermediate_model    13
                   non_dominant_model     2
                   dominant_model         1
Trees              intermediate_model     2
                   non_dominant_model     1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [194]:
summary_stats = df_sorted_topo_models_cols.groupby("Model").agg(
    Total_Models=("Model", "count"),
    Mean_Dominance_Count=("Dominates_Count", "mean"),
    Median_Dominance_Count=("Dominates_Count", "median"),
    Total_Dominated=("Dominated_Count", "sum"),
    Total_Dominating=("Dominates_Count", "sum"),
)

total_models = summary_stats[
    "Total_Models"
].sum()  # Get the total number of models across all categories
summary_stats["Dominance Proportion"] = summary_stats["Total_Dominating"] / total_models

<IPython.core.display.Javascript object>

In [195]:
summary_stats.round(2)

,Total_Models,Mean_Dominance_Count,Median_Dominance_Count,Total_Dominated,Total_Dominating,Dominance Proportion
Model,,,,,,
Linear Regression,1,17.00,17.0,1,17,0.85
Neural Networks,16,8.75,8.0,113,140,7.00
Trees,3,0.67,1.0,45,2,0.10


<IPython.core.display.Javascript object>

In [196]:
df_sorted_topo_models_grouped.T

RMSE_mean                                                    \
                      count      mean       std       min       25%       50%   
Model                                                                           
Linear Regression       1.0  1.659827       NaN  1.659827  1.659827  1.659827   
Neural Networks        16.0  1.863993  0.198701  1.601813  1.752573  1.801775   
Trees                   3.0  2.459605  0.157612  2.307811  2.378181  2.448552   

                                      MAE_mean            ... Dominated_Count  \
                        75%       max    count      mean  ...             75%   
Model                                                     ...                   
Linear Regression  1.659827  1.659827      1.0  1.210128  ...             1.0   
Neural Networks    1.902755  2.324786     16.0  1.441502  ...            10.0   
Trees              2.535503  2.622454      3.0  1.643724  ...            15.5   

                        Dominates_Count                                    \
                    max           count       mean       std   min    25%   
Model                                                                       
Linear Regression   1.0             1.0  17.000000       NaN  17.0  17.00   
Neural Networks    16.0            16.0   8.750000  5.422177   0.0   6.75   
Trees              16.0             3.0   0.666667  0.577350   0.0   0.50   

                                      
                    50%    75%   max  
Model                                 
Linear Regression  17.0  17.00  17.0  
Neural Networks     8.0  10.75  19.0  
Trees               1.0   1.00   1.0  

[3 rows x 56 columns]

<IPython.core.display.Javascript object>

### SCPM Analysis

In [197]:
cols = [
    # "Model",
    # "Model_bkp",
    "Model_bkp_2",
    "RMSE_mean",
    "MAE_mean",
    "MAPE_mean",
    "R2_mean",
    "SCPM",
    "Dominated_Count",
    "Dominates_Count",
    "Classification",
    "Plant",
]

<IPython.core.display.Javascript object>

#### SCPM Analysis by Project

In [198]:
df_sorted_scpm = df_sorted_topo.sort_values(by="SCPM").copy()

<IPython.core.display.Javascript object>

In [199]:
# (df_sorted_topo[cols].reset_index() == df_sorted_scpm[cols].reset_index()).all(axis=1)
df_sorted_scpm["topo_scmp_order_eq"] = df_sorted_topo.index == df_sorted_scpm.index

<IPython.core.display.Javascript object>

In [200]:
df_sorted_scpm[cols + ["topo_scmp_order_eq"]].round(2).head(5)

,Model_bkp_2,RMSE_mean,MAE_mean,MAPE_mean,R2_mean,SCPM,Dominated_Count,Dominates_Count,Classification,Plant,topo_scmp_order_eq
4,Transformer_1.0,1.60,1.20,2.86,0.91,-5.02,0,19,dominant_model,k,True
16,Linear Regression,1.66,1.21,2.94,0.90,-4.43,1,17,intermediate_model,k,False
12,GRU_14.0,1.64,1.26,2.96,0.90,-4.23,1,17,intermediate_model,k,False
0,BidirectionalGRU_1.0,1.74,1.31,3.11,0.89,-2.99,3,16,intermediate_model,k,True
13,LSTM_14.0,1.75,1.33,3.12,0.89,-2.78,4,13,intermediate_model,k,False


<IPython.core.display.Javascript object>

### Variable Grouping: CM-P-CS

In [201]:
df_results["Features"].unique()

array(['Chemical + Mineralogical', 'Chemical + Mineralogical + Physical',
       'Chemical + Mineralogical + Physical - Early CS'], dtype=object)

<IPython.core.display.Javascript object>

In [202]:
df_results_cm_p_cs = (
    df_results[df_results["Features"].eq("Chemical + Mineralogical + Physical")]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [203]:
df_results_cm_p_cs["Features"].unique()

array(['Chemical + Mineralogical + Physical'], dtype=object)

<IPython.core.display.Javascript object>

In [204]:
df_results_cm_p.shape

(20, 13)

<IPython.core.display.Javascript object>

### Time Series Split

In [205]:
df_results_cm_p_cs["Cross Validation"].unique()

array(['Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [206]:
df_results_cm_p_cs_tss = (
    df_results_cm_p_cs[df_results_cm_p_cs["Cross Validation"].eq("Time Series Split")]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [207]:
df_results_cm_p_cs_tss["Cross Validation"].unique()

array(['Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [208]:
df_results_cm_p_cs_tss.shape

(20, 13)

<IPython.core.display.Javascript object>

###### SCPM computation

In [209]:
df_results_cm_p_cs_tss = compute_scpm(df_results_cm_p_cs_tss)

<IPython.core.display.Javascript object>

##### Dominance Analysis

In [210]:
# Per project
dominance_dict = make_dominance_analysis(df_results_cm_p_cs_tss)
dominance_matrix_cm_p_cs_tss = dominance_dict["dominance_matrix"]
dominance_graph_cm_p_cs_tss = dominance_dict["dominance_graph"]
df_sorted_count = dominance_dict["df_sorted_count"]
df_sorted_topo = dominance_dict["df_sorted_topo"]

<IPython.core.display.Javascript object>

In [211]:
dominance_matrix_cm_p_cs_tss.shape, len(dominance_graph_cm_p_cs_tss)

((20, 20), 20)

<IPython.core.display.Javascript object>

In [212]:
df_sorted_topo["Classification"].value_counts()

Classification
intermediate_model    18
dominant_model         1
non_dominant_model     1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [213]:
df_sorted_topo.to_csv(
    "../../../../reports/results/local_models/ecics/209_k_dominance_analysis_cm_p_cs_tss_last_fold.csv",
    index=False,
)

<IPython.core.display.Javascript object>

In [214]:
df_sorted_topo.head(5)

,Company,Plant,Cross Validation,Features,Features_bkp,Model,Model_bkp,Model_bkp_2,Timesteps,RMSE_mean,MAE_mean,MAPE_mean,R2_mean,SCPM,Dominated_Count,Classification,Dominates_Count
16,209,k,Time Series Split,Chemical + Mineralogical + Physical,Chemical + Mineralogical + Physical,Linear Regression,Linear Regression,Linear Regression,NaN,0.858953,0.685311,1.619454,0.973503,-4.940424,0,dominant_model,19
18,209,k,Time Series Split,Chemical + Mineralogical + Physical,Chemical + Mineralogical + Physical,Trees,Random Forest,Random Forest,NaN,0.925393,0.713316,1.653255,0.969246,-4.448099,1,intermediate_model,18
0,209,k,Time Series Split,Chemical + Mineralogical + Physical,Chemical + Mineralogical + Physical,Neural Networks,BidirectionalGRU,BidirectionalGRU_1.0,1.0,1.011762,0.769299,1.808834,0.963237,-3.544779,2,intermediate_model,17
2,209,k,Time Series Split,Chemical + Mineralogical + Physical,Chemical + Mineralogical + Physical,Neural Networks,GRU,GRU_1.0,1.0,1.023714,0.780299,1.855113,0.962363,-3.367496,3,intermediate_model,15
5,209,k,Time Series Split,Chemical + Mineralogical + Physical,Chemical + Mineralogical + Physical,Neural Networks,BidirectionalGRU,BidirectionalGRU_7.0,7.0,1.023568,0.781068,1.817712,0.962257,-3.419729,3,intermediate_model,15


<IPython.core.display.Javascript object>

In [215]:
df_sorted_topo_cols = df_sorted_topo[
    [
        # "Model",
        # "Model_bkp",
        "Model_bkp_2",
        "RMSE_mean",
        "MAE_mean",
        "MAPE_mean",
        "R2_mean",
        "SCPM",
        "Dominated_Count",
        "Classification",
        "Dominates_Count",
        "Plant",
    ]
].round(2)

<IPython.core.display.Javascript object>

###### Dominant Models

In [216]:
# Top 5
df_sorted_topo_cols.sort_values(
    by=["Dominates_Count", "SCPM"], ascending=[False, True]
).iloc[0:5].round(2)

,Model_bkp_2,RMSE_mean,MAE_mean,MAPE_mean,R2_mean,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
16,Linear Regression,0.86,0.69,1.62,0.97,-4.94,0,dominant_model,19,k
18,Random Forest,0.93,0.71,1.65,0.97,-4.45,1,intermediate_model,18,k
0,BidirectionalGRU_1.0,1.01,0.77,1.81,0.96,-3.54,2,intermediate_model,17,k
5,BidirectionalGRU_7.0,1.02,0.78,1.82,0.96,-3.42,3,intermediate_model,15,k
2,GRU_1.0,1.02,0.78,1.86,0.96,-3.37,3,intermediate_model,15,k


<IPython.core.display.Javascript object>

In [217]:
df_sorted_topo_cols[df_sorted_topo_cols["Classification"].eq("dominant_model")].shape

(1, 10)

<IPython.core.display.Javascript object>

###### Intermediate models

In [218]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("intermediate_model")
].sort_values(by=["Dominates_Count", "SCPM"], ascending=[False, True]).iloc[:4]

,Model_bkp_2,RMSE_mean,MAE_mean,MAPE_mean,R2_mean,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
18,Random Forest,0.93,0.71,1.65,0.97,-4.45,1,intermediate_model,18,k
0,BidirectionalGRU_1.0,1.01,0.77,1.81,0.96,-3.54,2,intermediate_model,17,k
5,BidirectionalGRU_7.0,1.02,0.78,1.82,0.96,-3.42,3,intermediate_model,15,k
2,GRU_1.0,1.02,0.78,1.86,0.96,-3.37,3,intermediate_model,15,k


<IPython.core.display.Javascript object>

In [219]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("intermediate_model")
].shape

(18, 10)

<IPython.core.display.Javascript object>

###### non-dominant models

In [220]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("non_dominant_model")
].sort_values(by=["Dominated_Count", "SCPM"], ascending=[True, False]).iloc[:5]

,Model_bkp_2,RMSE_mean,MAE_mean,MAPE_mean,R2_mean,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
14,Transformer_14.0,2.06,1.68,4.11,0.85,10.25,19,non_dominant_model,0,k


<IPython.core.display.Javascript object>

In [221]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("non_dominant_model")
].shape

(1, 10)

<IPython.core.display.Javascript object>

###### Statistics per model type

In [222]:
df_sorted_topo_models_cols = df_sorted_topo[
    [
        "Model",
        # "Model_bkp",
        # "Model_bkp_2",
        "RMSE_mean",
        "MAE_mean",
        "MAPE_mean",
        "R2_mean",
        "SCPM",
        "Dominated_Count",
        "Dominates_Count",
        "Classification",
        "Plant",
    ]
]  # .round(2)

<IPython.core.display.Javascript object>

In [223]:
df_sorted_topo["Model"].value_counts()

Model
Neural Networks      16
Trees                 3
Linear Regression     1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [224]:
df_sorted_topo_models_grouped = df_sorted_topo_models_cols.groupby("Model").describe().T

<IPython.core.display.Javascript object>

In [225]:
df_sorted_topo_models_cols = df_sorted_topo[
    [
        "Model",
        "Dominated_Count",
        "Dominates_Count",
        "Classification",
    ]
]

<IPython.core.display.Javascript object>

In [226]:
(
    df_sorted_topo_models_cols.groupby("Model")[["Classification"]]
    .value_counts(normalize=True)
    .to_frame()
    * 100
).round(2)

proportion
Model             Classification                
Linear Regression dominant_model          100.00
Neural Networks   intermediate_model       93.75
                  non_dominant_model        6.25
Trees             intermediate_model      100.00

<IPython.core.display.Javascript object>

In [227]:
df_sorted_topo_models_cols.groupby("Model").describe().round(2).T

Model                  Linear Regression  Neural Networks  Trees
Dominated_Count count                1.0            16.00   3.00
                mean                 0.0             9.69   9.33
                std                  NaN             5.49   7.23
                min                  0.0             2.00   1.00
                25%                  0.0             5.75   7.00
                50%                  0.0             9.00  13.00
                75%                  0.0            14.00  13.50
                max                  0.0            19.00  14.00
Dominates_Count count                1.0            16.00   3.00
                mean                19.0             8.56   9.00
                std                  NaN             5.50   7.94
                min                 19.0             0.00   3.00
                25%                 19.0             3.00   4.50
                50%                 19.0             9.50   6.00
                75%                 19.0            13.25  12.00
                max                 19.0            17.00  18.00

<IPython.core.display.Javascript object>

In [228]:
# df_sorted_topo_models_cols

<IPython.core.display.Javascript object>

In [229]:
df_sorted_topo_models_cols.groupby("Model")["Classification"].describe()

,count,unique,top,freq
Model,,,,
Linear Regression,1,1,dominant_model,1
Neural Networks,16,2,intermediate_model,15
Trees,3,1,intermediate_model,3


<IPython.core.display.Javascript object>

In [230]:
df_sorted_topo_models_cols.groupby("Model")[["Classification"]].value_counts()

Model              Classification    
Linear Regression  dominant_model         1
Neural Networks    intermediate_model    15
                   non_dominant_model     1
Trees              intermediate_model     3
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [231]:
summary_stats = df_sorted_topo_models_cols.groupby("Model").agg(
    Total_Models=("Model", "count"),
    Mean_Dominance_Count=("Dominates_Count", "mean"),
    Median_Dominance_Count=("Dominates_Count", "median"),
    Total_Dominated=("Dominated_Count", "sum"),
    Total_Dominating=("Dominates_Count", "sum"),
)

total_models = summary_stats[
    "Total_Models"
].sum()  # Get the total number of models across all categories
summary_stats["Dominance Proportion"] = summary_stats["Total_Dominating"] / total_models

<IPython.core.display.Javascript object>

In [232]:
summary_stats.round(2)

,Total_Models,Mean_Dominance_Count,Median_Dominance_Count,Total_Dominated,Total_Dominating,Dominance Proportion
Model,,,,,,
Linear Regression,1,19.00,19.0,0,19,0.95
Neural Networks,16,8.56,9.5,155,137,6.85
Trees,3,9.00,6.0,28,27,1.35


<IPython.core.display.Javascript object>

In [233]:
df_sorted_topo_models_grouped.T

RMSE_mean                                                    \
                      count      mean       std       min       25%       50%   
Model                                                                           
Linear Regression       1.0  0.858953       NaN  0.858953  0.858953  0.858953   
Neural Networks        16.0  1.329495  0.341041  1.011762  1.065938  1.197150   
Trees                   3.0  1.211769  0.248461  0.925393  1.132682  1.339971   

                                      MAE_mean            ... Dominated_Count  \
                        75%       max    count      mean  ...             75%   
Model                                                     ...                   
Linear Regression  0.858953  0.858953      1.0  0.685311  ...             0.0   
Neural Networks    1.471691  2.060575     16.0  1.038181  ...            14.0   
Trees              1.354957  1.369943      3.0  0.989301  ...            13.5   

                        Dominates_Count                                       \
                    max           count     mean       std   min   25%   50%   
Model                                                                          
Linear Regression   0.0             1.0  19.0000       NaN  19.0  19.0  19.0   
Neural Networks    19.0            16.0   8.5625  5.501136   0.0   3.0   9.5   
Trees              14.0             3.0   9.0000  7.937254   3.0   4.5   6.0   

                                
                     75%   max  
Model                           
Linear Regression  19.00  19.0  
Neural Networks    13.25  17.0  
Trees              12.00  18.0  

[3 rows x 56 columns]

<IPython.core.display.Javascript object>

### SCPM Analysis

In [234]:
cols = [
    # "Model",
    # "Model_bkp",
    "Model_bkp_2",
    "RMSE_mean",
    "MAE_mean",
    "MAPE_mean",
    "R2_mean",
    "SCPM",
    "Dominated_Count",
    "Dominates_Count",
    "Classification",
    "Plant",
]

<IPython.core.display.Javascript object>

#### SCPM Analysis by Project

In [235]:
df_sorted_scpm = df_sorted_topo.sort_values(by="SCPM").copy()

<IPython.core.display.Javascript object>

In [236]:
# (df_sorted_topo[cols].reset_index() == df_sorted_scpm[cols].reset_index()).all(axis=1)
df_sorted_scpm["topo_scmp_order_eq"] = df_sorted_topo.index == df_sorted_scpm.index

<IPython.core.display.Javascript object>

In [237]:
df_sorted_scpm[cols + ["topo_scmp_order_eq"]].round(2).head(5)

,Model_bkp_2,RMSE_mean,MAE_mean,MAPE_mean,R2_mean,SCPM,Dominated_Count,Dominates_Count,Classification,Plant,topo_scmp_order_eq
16,Linear Regression,0.86,0.69,1.62,0.97,-4.94,0,19,dominant_model,k,True
18,Random Forest,0.93,0.71,1.65,0.97,-4.45,1,18,intermediate_model,k,True
0,BidirectionalGRU_1.0,1.01,0.77,1.81,0.96,-3.54,2,17,intermediate_model,k,True
5,BidirectionalGRU_7.0,1.02,0.78,1.82,0.96,-3.42,3,15,intermediate_model,k,False
2,GRU_1.0,1.02,0.78,1.86,0.96,-3.37,3,15,intermediate_model,k,False


<IPython.core.display.Javascript object>